
# Project Final Report


# 0. Background information

Minecraft is a video game played by a wide range of people. PLAI, a computer science research group at UBC, aims to collect data about the ways in which individuals play Minecraft by recording their in-game actions. This report seeks to assist PLAI with their recruitment efforts in order to ensure it is optimally targeted. Thus, PLAI has provided us with a dataset collected through an online survey, which contains the name, experience, email, playtime, gender, and age of players. Starting from a broad initial question, we conducted exploratory analysis to narrow it to a specific research question that we hope will yield insightful results that can help PLAI optimally recruit players for their research.

# 1. Data Description

## 1.1 Dataset Overview
In this project, we will be working with the file **players.csv**, which contains one row per unique player and several demographic and gameplay-related features.

- **Number of observations:** 196
- **Number of variables:** 7

## 1.2 Variable Summary Table

| Variable       | Type      | Description | Missing | Unique Values |
|----------------|-----------|-------------|---------|---------------|
| experience     | categorical | Player’s self-reported gaming experience | 0 | 5 |
| subscribe      | boolean | Whether the player subscribed to the newsletter | 0 | 2 |
| hashedEmail    | identifier | Unique hashed player ID | 0 | 196 |
| played_hours   | numeric | Total hours played on the server | 0 | 43 |
| name           | text | Player name | 0 | 196 |
| gender         | categorical | Gender identity | 0 | 7 |
| Age            | numeric | Player age in years | 2 | 32 |


## 1.3 Numeric Summary Statistics

| Variable     |Count | Mean | Std Dev | Min | Median | Max |
|--------------|------|----------|------|--------|------|----|
| played_hours |  196   |    5.85      |    28.36  |   0.00     |  0.10    |  223.1  |
| Age          |  194    |   21.14       |   7.39   |   9.00     |   19.00   |  58  |

## 1.4 Notes and Potential Data Issues
- Age contains 2 missing values.  
- `played_hours` is heavily skewed with a few large outliers. Many players have close to 0 or very small play time.  
- There may be volunteer/self-selection bias in the dataset.  
- `name` contains personal information and should be excluded from all analysis.
- Subscription classes are imbalanced (about 73% subscribed).
- Experience appears to be ordinal and we should preserve this




## 1.5 Overall summary

The **players.csv** file contains 196 unique players and 7 variables. Key numeric variables are **played_hours** (total play time per player) and **Age** (two missing values). **Subscribe** is a boolean indicating newsletter signup (144 True, 52 False). Categorical variables include **experience** (5 levels) and **gender** (7 categories). Computer checks found no duplicate **hashedEmail** identifiers. **played_hours** is highly skewed (mean 5.85 hours, median 0.10 hours, max 223.1). Potential limitations include class imbalance for **subscribe**, small counts in some **gender** categories, and volunteer bias inherent to the data collection method.

# 2. Questions

## 2.1 Broad Question
**What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?**

## 2.2 Specific Question
**Can player demographic attributes (`Age`, `gender`) and gameplay characteristics (`experience`, `played_hours`) predict whether a player subscribes to the game-related newsletter?**

This question is appropriate because `players.csv` contains all required explanatory variables and a clear binary response variable (`subscribe`). This will be a standard classification problem. To apply models seen in class, we will likely need to scale the numeric variables, impute the missing values in the age variable, and encode the categorical variables to numeric form.

# 3. Exploratory Data Analysis (EDA)
## 3.1 Load Data

In [ ]:
library(tidyverse)
library(tidymodels)

In [ ]:
url <- "players (1).csv"
players <- read_csv(url)
head(players)

## 3.2 Basic Data Wrangling

In [ ]:
library(tidyverse)

players_tidy <- players |>
  mutate(
    subscribe = as.logical(subscribe),
    gender = as.factor(gender),
    experience = as.factor(experience)
  )

head(players_tidy)

Note: We did not change name to factor as that will likely be dropped


## 3.3 Mean and Summary statistics of Quantitative Variables

In [ ]:
players_tidy |>
  summarize(
    mean_played_hours = mean(played_hours, na.rm = TRUE),
    median_played_hours = median(played_hours, na.rm = TRUE),
    mean_age = mean(Age, na.rm = TRUE),
    median_age = median(Age, na.rm = TRUE)
  )

In [ ]:
summary_players <- summary (players)
summary_players

## 3.4 Exploratory Visualizations


## Class Imbalance Visulaizations



### Class Imbalance for Subscribe


Figure 1.1 shows a class imbalance for subscribed vs not subscribed. Subscribed has 144 and not subscribed has 52 data points.

In [ ]:
#Creating a graph for number of subscribed vs not subscribed
subscribe_graph <- players_tidy |>
    ggplot (aes (x= subscribe, fill = subscribe)) +
    geom_bar (color = "black") +
    xlab ("Subscribed (True or False)") +
    ylab ("Subscribed or Not Subscribed Count") +
    ggtitle ("Figure 1.1") +
    labs (subtitle = "Number of people subscribed or not subscribed in barplot") +
    scale_fill_manual(values = c("steelblue", "darkorange")) +
    theme (text = element_text (size = 15)) 
    options (repr.plot.width = 10, repr.plot.height = 8)
subscribe_graph

Figure 1.2 shows the number of players playing minecraft for each gender. Male has the most amount of players and then it is female. All other catergories have a lot less players demonstrating that there is a inbalance (Figure 1.2).

In [ ]:
ggplot(players_tidy, aes(x = gender)) +
  geom_bar(alpha = 0.5, position = "identity") +
  coord_cartesian(ylim = c(0, 50)) +
  labs(
    title = "Figure 1.2",
    x = "Gender",
    subtitle = "Number of players by gender"
  ) +
  theme_minimal(base_size = 12)

For figure 1.3 there is the most players playing for around 0 hour point. After that for subscribed and not subscribed the number of players playing is a lot less (Figure 1.3). The subscribed players catergory seems to have some players still playing after the 0 hour mark and even has some players playing for over 200 hours. For not subscribed category the players do not play for longer than around the 0 hours mark (Figure 1.3). This indicates that the subscribed players are playing minecraft a lot more that the not subscribed.

In [ ]:
ggplot(players_tidy, aes(x = played_hours, fill = subscribe)) +
  geom_histogram(position = "identity", alpha = 0.5, bins = 30) +
  labs(title = "Figure 1.3", subtitle = "Distribution of Played Hours by Subscription Status",
       x = "Played Hours (hours)",
       y = "Count",
       fill = "Subscribed",
      ) +
theme (text = element_text (size = 15)) +
  theme_minimal()


# Figure 1.4 description
The figure shows the age distribution of players by subscription status.

Overall, both subscribed and non-subscribed players are heavily concentrated in the 18–25 age range. Non-subscribed players also appear in this range but in noticeably smaller numbers, so their bars are partially overshadowed in the overlapping histogram.

Above age 30, both groups have very few players, appearing only as scattered counts.
This pattern suggests that the majority of subscribers are young players, while non-subscribers are fewer and more spread out across ages.

In [ ]:
ggplot(players_tidy, aes(x = Age, fill = subscribe)) +
  geom_histogram(position = "identity", alpha = 0.6, bins = 20) +
  labs(title = "Figure 1.4",
       x = "Age (years)",
       y = "Count", subtitle = "Age Distribution by Subscription Status",
       fill = "Subscribed") +
theme (text = element_text (size = 15)) +
  theme_minimal()


# Figure 1.5 description
Figure 1.5 shows the subscription rate across different experience levels. Overall, the proportion of subscribed players is consistently high in all groups, generally ranging from about 70% to 80%. Beginner and Regular players have the highest subscription rates, both close to or slightly above 0.8. Amateur and Pro players show slightly lower rates, but still around 0.7. Veteran players have the lowest subscription proportion among the five groups, though the difference is relatively small.

This pattern suggests that subscription is common across all experience levels.

In [ ]:
players_tidy |>
  group_by(experience) |>
  summarize(prop_subscribe = mean(subscribe)) |>
  ggplot(aes(x = experience, y = prop_subscribe)) +
  geom_col() +
  labs(title = "Figure 1.5",
       x = "Experience Level",
       y = "Proportion Subscribed", subtitle = "Subscription Rate by Experience Level")


## 3.5 Insights from EDA

- The visualization supports our previous discussion that playing hours are highly skewed, with many low-playtime users and a few players with very large playtimes. Although an interesting trend, we can see with the distribution of played hours by subscription status is that nearly all of the non-subscribed players have playtime around 0, while players with more playtime are almost always subscribed. This trend could be beneficial to our prediction question.
- The age visualization shows that our dataset is dominated by young adults. Looking at the distribution of age by subscription status, we see heavy overlap between the ages of players who are subscribed and non-subscribers. But all young players (age < 16) are subscribers, which could make age a beneficial variable in our prediction question.
- We can see that subscription rates are relatively similar across experience levels (around 0.7). However, this does not mean that the experience level variable is useless in our prediction question, as it could be helpful in conjunction with another variable to a predictive model trying to answer our question.

# 4. Methods and Plan

## 4.1 Proposed Method
We propose using KNN to predict a player's subscription status based on their demographic and gameplay characteristics.

## 4.2 Why This Method is Appropriate
- The problem is a classification problem (cannot use linear regression)
- KNN is a good option because it predicts based on similar data points. From our EDA, we have seen some similarity in the subscription status as some of the variables change. For example, all players with high playtime are subscribers. Another example is that almost all young players are subscribers.
- KNN can handle some of the skewed variables like played_hours, as most players would be far away from the players with outlier playtime. Thus, the outliers would not be able to heavily influence predictions.

## 4.3 Required Assumptions
- Similar points should have similar outcomes.
- Numeric representation of all variables. This is pretty achievable with encoding categorical columns.
- Scaling will be needed for the KNN model to be effective.
- No irrelevant or noisy features.
- The dataset should be reasonably balanced. If one class is far more frequent in the dataset, then it will obviously comprise more neighbors and be predicted more often.

## 4.4 Potential Limitations/Weaknesses
- Class imbalance in subscription status may heavily affect predictive performance.
- Might not be able to capture linear relationships between variables and the target.

## 4.5 Model Evaluation Plan
- Split data into training (70%) and test (30%) sets before any preprocessing or training.
- Create a majority classifier model to get baseline statistics.
- Perform n-fold cross-validation (n=5 or 10) on the training set, while hyperparameter optimizing k for the KNN model.
- Evaluate the model using f1 score, precision, and recall due to class imbalance. We will give 
- Compare the model with the baseline statistics.




# 5. Model Building

## 5.0 Data transformations overview


The KNN model requires numeric columns. It also needs the columns to be scaled so that all columns have equal weight during classification. Keeping that in mind, we will apply the following transformations to the dataset:

- The experience column should be ordinally encoded, as there is a natural hierarchy within it (Amateur = 0, Beginner = 1, Regular = 2, Pro = 3, Veteran = 4)
- The gender column should be one-hot encoded. This would entail splitting the column into multiple columns, one for each category. If an observation has a specific category from the original column, then the corresponding new column will have a value of 1, otherwise, it will have a value of 0.
- We should drop the name and hashedEmail columns, as they won't provide any meaningful contribution to our model. It is reasonable to say that they don't encode any patterns for the model to learn.
- We should convert the subscribe column into a factor

We can apply these transformations to the entire dataset, as they are uniform rules that don't leak any information. The future training set will not learn anything from the future testing set if we apply these transformations (preserving the golden rule).

Note: We will scale and center all the columns after splitting the dataset into training and testing sets, and do it separately for each cross-validation iteration.

## 5.0.1 One-hot encoding gender column

In [ ]:
one_hot_recipe <- recipe(~ ., data = players) |> 
                  step_dummy(gender, one_hot = TRUE)

processing_players <- prep(one_hot_recipe) |> 
                   bake(new_data = NULL)

head(processing_players)

## 5.0.2 Ordinally encoding experience column

In [ ]:
## Note: Only run this cell once, running it a second time will apply it again to the experience column, which won't have the categories anymore (leading to NA being assigned to all rows)
processing_players$experience <- factor(
  processing_players$experience,
  levels = c("Amateur", "Beginner", "Regular", "Pro", "Veteran"),
  labels = c(0, 1, 2, 3, 4)
)

In [ ]:
head(processing_players)


## 5.0.3 Converting subscribe to factor type and dropping uneeded columns

In [ ]:
processing_players$hashedEmail <- NULL
processing_players$name <- NULL
processing_players$subscribe <- as_factor(processing_players$subscribe)

head(processing_players)

The dataset is now ready to be split into training and testing sets.

## 5.1 Creating training and testing sets

We should set the seed first to ensure reproducibility. Since the subscribe column is imbalanced, we need to stratify by it during the split to ensure equal representation of both subscribed and unsubscribed players in both the training and test splits.

In [ ]:
set.seed(123)

players_split <- initial_split(processing_players, prop = 0.7, strata = subscribe)
players_test <- testing(players_split)
players_training <- training(players_split)

## 5.2 Baseline model

In order to later evaluate our KNN model against some baseline metrics, we will first build a simple baseline model using null_model() from tidymodels. This model will look only at the subscribe column (no need to scale predictors yet) and will predict the "majority class" for all players. In this case, that would mean predicting every player as subscribed.

In [ ]:
baseline_specification <- null_model(mode = "classification") |>
                          set_engine("parsnip")

baseline_fit <- fit(baseline_specification, subscribe ~ ., data = players_training)
preds_training <- predict(baseline_fit, players_training)
preds_testing <- predict(baseline_fit, players_test)


As previously discussed, due to class imbalance in the subscribe column, we cannot rely on accuracy to evaluate the model. We will use precision, recall and f1 score. 

In [ ]:
eval_players_training <- players_training
eval_players_training <- mutate(players_training, pred = preds$.pred_class)

# Discussion

- summarize what you found
- discuss whether this is what you expected to find
- discuss what impact could such findings have
- discuss what future questions could this lead to


# References


- You may include references if necessary, as long as they all have a consistent citation style.